In [1]:
# --- Imports ---
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import preprocess as pre
import time

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb

In [2]:
# --- Data Preprocessing --- 
start = time.time()
pre.ais_trainCleanup("../CSV/big_files/ais_train.csv", "ais_train_processed.csv")
end = time.time()

print(end-start)

c:\Users\marti\Desktop\ML-master\Kuking\preprocess.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boat["time"] = boat["time"].astype("datetime64[ns]")
c:\Users\marti\Desktop\ML-master\Kuking\preprocess.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boat["drift"] = boat["cog"] - boat["heading"]
c:\Users\marti\Desktop\ML-master\Kuking\preprocess.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

4.894385099411011


In [8]:
data = pd.read_csv("../CSV/big_files/ais_train_processed.csv", sep="|")
data.drop(columns=["portId"], inplace=True)

X = data.copy()
y = data.drop(columns=["time", "vesselId"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train = pd.get_dummies(X_train, columns=["vesselId"], drop_first=True)
X_test = pd.get_dummies(X_test, columns=["vesselId"], drop_first=True)

#More data BS
for col in X_train:
    if col not in X_test:
        X_test[col] = False

X_train = X_train.reindex(sorted(X_train.columns), axis=1)
X_test = X_test.reindex(sorted(X_test.columns), axis=1)



,at_port,cog,drift,etaRaw,heading,latitude,longitude,navstat,rot,sog,...,vesselId_clh682fbq000qghxd3vn5xo01,vesselId_clh682fbq000rghxd2u75zdbn,vesselId_clh6aqawa0000gh0z7nc8e6nd,vesselId_clh6aqawa0001gh0zmijpuho1,vesselId_clh6aqawa0002gh0zypfa5dut,vesselId_clh6aqawa0003gh0zu0aznvt2,vesselId_clh6aqawa0004gh0z12aogec9,vesselId_clh6aqawa0005gh0z64y4xyip,vesselId_clh6aqawa0006gh0zje911dl3,vesselId_clh6aqawa0007gh0z9h6zi9bo
0,False,284.0,196.0,1704841200,88,-34.74370,-57.85130,0,0,0.7,...,False,False,False,False,False,False,False,False,False,False
1,False,109.6,-237.4,1735502400,347,8.89440,-79.47939,1,-6,0.0,...,False,False,False,False,False,False,False,False,False,False
2,False,111.0,-1.0,1704186000,112,39.19065,-76.47567,0,0,11.0,...,False,False,False,False,False,False,False,False,False,False
3,False,96.4,-45.6,1735675200,142,-34.41189,151.02067,1,0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,False,214.0,-1.0,1706184000,215,35.88379,-5.91636,0,0,19.7,...,False,False,False,False,False,False,False,False,False,False


In [10]:
def evaluate(model: str, y_pred):

    n = len(y_test)
    p = X_test.shape[1]

    mse_latitude = mean_squared_error(y_test, y_pred)
    mse_longitude = mean_squared_error(y_test, y_pred)
    r2_latitude = r2_score(y_test, y_pred)
    r2_latitude_adj = 1 - (1 - r2_latitude) * ((n - 1) / (n - p - 1))
    r2_longitude = r2_score(y_test, y_pred)
    r2_longitude_adj = 1 - (1 - r2_longitude) * ((n - 1) / (n - p - 1))


    print(f"---- {model} Metrics ----")
    print(f"Mean Squared Error (Latitude): {mse_latitude:.4f}")
    print(f"R-squared (Latitude): {r2_latitude:.4f}, Adjusted R-squared (Latitude): {r2_latitude_adj: .4f}")
    print(f"Mean Squared Error (Longitude): {mse_longitude:.4f}")
    print(f"R-squared (Longitude): {r2_longitude:.4f}, Adjusted R-squared (Latitude): {r2_longitude_adj: .4f}")


In [ ]:
# --- LINEAR REGRESSION ---
linreg = LinearRegression()
linreg.fit(X_train, y_train)
linreg_pred = linreg.predict(X_test)

#Metrics
evaluate("Linear Regression", linreg_pred)